In [9]:
!pip install sqlalchemy==1.3.9


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [10]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [11]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [12]:
!pip install -q pandas==1.1.5



In [13]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [14]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")




Now write and execute SQL queries to solve the assignment tasks.

##### Display the names of the unique launch sites  in the space mission


In [15]:
%sql select distinct(Launch_Site) from SPACEXTBL


 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40
None





#####  Display 5 records where launch sites begin with the string 'CCA' 


In [16]:
%sql select* from SPACEXTBL where Launch_Site like'%CCA%' limit 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt







##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [21]:
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTBL where Customer like '%CRS%'

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
48213.0







##### We display average payload mass carried by booster version F9 v1.1


In [22]:
%sql select avg(payload_mass__kg_ ) from spacextbl where booster_version like '%F9 v1.1%'

 * sqlite:///my_data1.db
Done.


avg(payload_mass__kg_ )
2534.6666666666665




##### We list the date when the first succesful landing outcome in ground pad was acheived.




In [31]:
%sql select max("Date") from spacextbl where landing_outcome like '%success%'

 * sqlite:///my_data1.db
Done.


"max(""Date"")"
30/10/2017




##### We list the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [32]:
%sql select distinct(booster_version) from spacextbl  where (payload_mass__kg_ >= 4000 and payload_mass__kg_<=6000)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 v1.1
F9 v1.1 B1011
F9 v1.1 B1014
F9 v1.1 B1016
F9 FT B1020
F9 FT B1022
F9 FT B1026
F9 FT B1030
F9 FT B1021.2
F9 FT B1032.1







##### We list the total number of successful and failure mission outcomes


In [33]:
%sql select mission_outcome, count(mission_outcome) from spacextbl group by mission_outcome

 * sqlite:///my_data1.db
Done.


Mission_Outcome,count(mission_outcome)
None,0
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1






##### We list the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [34]:
%sql select booster_version from spacextbl where payload_mass__kg_=(select max(payload_mass__kg_) from spacextbl)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6





##### We list the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.**


In [36]:
%sql SELECT substr(Date, 4, 2) as Month, booster_version, Landing_outcome, Launch_Site FROM SPACEXTBL WHERE Landing_outcome like '%drone%'

 * sqlite:///my_data1.db
Done.


Month,Booster_Version,Landing_Outcome,Launch_Site
10,F9 v1.1 B1012,Failure (drone ship),CCAFS LC-40
04,F9 v1.1 B1015,Failure (drone ship),CCAFS LC-40
06,F9 v1.1 B1018,Precluded (drone ship),CCAFS LC-40
01,F9 v1.1 B1017,Failure (drone ship),VAFB SLC-4E
04,F9 FT B1020,Failure (drone ship),CCAFS LC-40
08,F9 FT B1021.1,Success (drone ship),CCAFS LC-40
06,F9 FT B1022,Success (drone ship),CCAFS LC-40
05,F9 FT B1023.1,Success (drone ship),CCAFS LC-40
06,F9 FT B1024,Failure (drone ship),CCAFS LC-40
08,F9 FT B1026,Success (drone ship),CCAFS LC-40







##### We rank the  count of  successful landing_outcomes between the date 04-06-2010 and 20-03-2017 in descending order.


In [40]:
%sql SELECt Landing_outcome, COUNT(*) as Quantity FROM SPACEXTBL WHERE Date BETWEEN '04-06-2010' AND '20-03-2017'

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Quantity
Failure (parachute),56
